# MyUpchar blogs extraction


## 1. Article scrapper code:


In [2]:
import requests
from bs4 import BeautifulSoup
import json, os


def clean_text_with_links(tag):
    """Rebuild paragraph text while preserving <a> tag text with space."""
    parts = []
    for child in tag.descendants:
        if child.name == "a":
            text = child.get_text(strip=True)
            parts.append(text)
        elif isinstance(child, str):
            parts.append(child)
    return "".join(parts).strip()


def extract_intro_content(content_body_div):
    """Extracts the content before sections (main body / intro)."""
    intro = []
    for child in content_body_div.children:
        if getattr(child, "name", None) == "p":
            strong = child.find("strong")
            if strong and strong.text.strip() == child.text.strip():
                intro.append("### " + strong.get_text(strip=True))
            else:
                intro.append(clean_text_with_links(child))
        elif getattr(child, "name", None) == "ul":
            for li in child.find_all("li"):
                intro.append("• " + clean_text_with_links(li))
    return intro


def scrape_myupchar_article(url):
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")

    title = soup.find("h1").get_text(strip=True) if soup.find("h1") else "No Title"
    content = {"url": url, "title": title, "intro": [], "sections": []}

    # === Extract Main Body (before sections)
    intro_div = soup.find("div", id="content-body")
    if intro_div:
        content["intro"] = extract_intro_content(intro_div)

    # === Extract Structured Sections
    for section in soup.select(".category_blue_backdiv"):
        heading_tag = section.find("h2")
        body_tag = section.find("div", class_="description")

        section_title = (
            heading_tag.get_text(strip=True) if heading_tag else "No Heading"
        )
        section_text = []

        if body_tag:
            for child in body_tag.children:
                if child.name == "p":
                    strong = child.find("strong")
                    if strong and strong.text.strip() == child.text.strip():
                        section_text.append("### " + strong.get_text(strip=True))
                    else:
                        section_text.append(clean_text_with_links(child))
                elif child.name == "ul":
                    for li in child.find_all("li"):
                        section_text.append("• " + clean_text_with_links(li))
                elif child.name == "div":
                    for subchild in child.children:
                        if subchild.name == "p":
                            section_text.append(clean_text_with_links(subchild))
                        elif subchild.name == "ul":
                            for li in subchild.find_all("li"):
                                section_text.append("• " + clean_text_with_links(li))

        content["sections"].append({"heading": section_title, "body": section_text})

    return content

### 1.1. Save as Markdown


In [3]:
def save_as_markdown(article_data, filename):
    with open(filename, "w", encoding="utf-8") as md:
        md.write(f"# {article_data['title']}\n")
        md.write(f"_Source: [{article_data['url']}]({article_data['url']})_\n\n")

        if article_data["intro"]:
            md.write("## Introduction\n")
            for line in article_data["intro"]:
                if line.startswith("### "):
                    md.write(f"{line}\n")
                elif line.startswith("• "):
                    md.write(f"- {line[2:]}\n")
                else:
                    md.write(f"{line}\n")
            md.write("\n")

        for sec in article_data["sections"]:
            md.write(f"## {sec['heading']}\n")
            for line in sec["body"]:
                if line.startswith("### "):
                    md.write(f"{line}\n")
                elif line.startswith("• "):
                    md.write(f"- {line[2:]}\n")
                else:
                    md.write(f"{line}\n")
            md.write("\n")

## 2. Bulk Article URL Scrapper


#### Tool 1: Extract Links from Category page


In [ ]:
from bs4 import BeautifulSoup


def extract_blog_urls(html_path, base_url="https://www.myupchar.com"):
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(html_path, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")

    links = set()

    for h4 in soup.find_all("h4"):
        a_tag = h4.find("a", href=True)
        if a_tag:
            href = a_tag["href"]
            links.add(base_url + href.strip())

    return sorted(links)


urls = extract_blog_urls(input("Paste category page URL here: "))

with open("myupchar_urls.txt", "w", encoding="utf-8") as f:
    for url in urls:
        f.write(url + "\n")

print(f"✅ Extracted {len(urls)} article URLs from <h4> tags.")

✅ Extracted 21 article URLs from <h4> tags.


#### Tool 2: Bulk Scrap articles from saved links


In [5]:
import os
import time
import json
import random

# Reuse the existing scrape_myupchar_article() and save_as_markdown() functions
# Make sure those are defined/imported in this script
import os
import json


def safe_path_from_url(url):
    """Convert the URL path into a folder path (no leading slash)."""
    path = url.replace("https://www.myupchar.com/", "").strip("/")
    return path  # e.g., "en/disease/acidity"


def save_all_formats(article):
    relative_path = safe_path_from_url(article["url"])  # e.g., en/disease/acidity
    base_name = os.path.basename(relative_path)  # e.g., acidity

    # Paths
    txt_path = os.path.join("blogs", relative_path + ".txt")
    json_path = os.path.join("jsonblogs", relative_path + ".json")
    md_path = os.path.join("markdownblogs", relative_path + ".md")

    # Create directories
    os.makedirs(os.path.dirname(txt_path), exist_ok=True)
    os.makedirs(os.path.dirname(json_path), exist_ok=True)
    os.makedirs(os.path.dirname(md_path), exist_ok=True)

    # Save as .txt
    with open(txt_path, "w", encoding="utf-8") as f:
        f.write(f"Title: {article['title']}\n")
        f.write(f"URL: {article['url']}\n\n")
        f.write("== Introduction ==\n")
        for line in article["intro"]:
            f.write(f"{line}\n")
        for sec in article["sections"]:
            f.write(f"\n== {sec['heading']} ==\n")
            for line in sec["body"]:
                f.write(f"{line}\n")

    # Save as .json
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(article, f, indent=4, ensure_ascii=False)

    # Save as .md
    save_as_markdown(article, md_path)

    print(f"✅ Saved:\n  TXT: {txt_path}\n  JSON: {json_path}\n  MD: {md_path}")


def bulk_scrape(url_list, delay_range=(2, 5)):
    failed_urls = []
    success_index = []

    for idx, url in enumerate(url_list, start=1):
        print(f"[{idx}/{len(url_list)}] Scraping: {url}")

        try:
            # Use URL path to construct relative storage path
            relative_path = safe_path_from_url(url)  # e.g., en/disease/acidity
            base_name = os.path.basename(relative_path)

            json_path = os.path.join("jsonblogs", relative_path + ".json")

            # ✅ Skip early if already exists
            if os.path.exists(json_path):
                print(f"⚠️ Skipped (already scraped): {relative_path}")
                continue

            article = scrape_myupchar_article(url)
            save_all_formats(article)
            print(f"✅ Success: {article['title']}")

            success_index.append(
                {
                    "title": article["title"],
                    "slug": base_name,
                    "url": article["url"],
                    "path": relative_path,
                    "files": {
                        "txt": f"blogs/{relative_path}.txt",
                        "json": f"jsonblogs/{relative_path}.json",
                        "md": f"markdownblogs/{relative_path}.md",
                    },
                }
            )

            # ✅ Only wait if we actually scraped
            time.sleep(random.uniform(*delay_range))

        except Exception as e:
            print(f"❌ Failed: {url}\n   Reason: {e}")
            failed_urls.append(url)

    # Save failed URLs
    if failed_urls:
        with open("failed_urls.txt", "w", encoding="utf-8") as f:
            f.writelines(url + "\n" for url in failed_urls)
        print("⚠️ Failed URLs saved to 'failed_urls.txt'")

    # Save master index
    with open("master_index.json", "w", encoding="utf-8") as f:
        json.dump(success_index, f, indent=4, ensure_ascii=False)
    print("📚 Master index saved to 'master_index.json'")

    print(f"\n🟢 Finished: {len(success_index)} success, {len(failed_urls)} failed.")


# Load main list of URLs
with open("myupchar_urls.txt", "r", encoding="utf-8") as f:
    all_urls = [line.strip() for line in f if line.strip()]

# Load failed URLs (if resume is enabled)
if os.path.exists("failed_urls.txt"):
    with open("failed_urls.txt", "r", encoding="utf-8") as f:
        failed = [line.strip() for line in f if line.strip()]
    all_urls.extend(failed)

# Remove duplicates
all_urls = list(set(all_urls))

# Start scraper
bulk_scrape(all_urls)

[1/21] Scraping: https://www.myupchar.com/pregnancy/which-side-is-baby-boy-in-womb
✅ Saved:
  TXT: blogs/pregnancy/which-side-is-baby-boy-in-womb.txt
  JSON: jsonblogs/pregnancy/which-side-is-baby-boy-in-womb.json
  MD: markdownblogs/pregnancy/which-side-is-baby-boy-in-womb.md
✅ Success: प्रेगनेंसी में बेबी बॉय किस साइड होता है? - Which side is baby boy side in womb in Hindi
[2/21] Scraping: https://www.myupchar.com/sexual-health/masturbation-hastmaithun-in-hindi/masturbating-while-pregnant-is-safe
✅ Saved:
  TXT: blogs/sexual-health/masturbation-hastmaithun-in-hindi/masturbating-while-pregnant-is-safe.txt
  JSON: jsonblogs/sexual-health/masturbation-hastmaithun-in-hindi/masturbating-while-pregnant-is-safe.json
  MD: markdownblogs/sexual-health/masturbation-hastmaithun-in-hindi/masturbating-while-pregnant-is-safe.md
✅ Success: क्या गर्भावस्था में हस्तमैथुन कर सकते हैं? - Is masturbation during pregnancy safe in Hindi
[3/21] Scraping: https://www.myupchar.com/pregnancy/diet/what-should-